## Versionamento com: 
- python 
- gitpython
- jupyter

### requisitos
- python 3.11
- pip install gitpython

In [80]:
# Bibliotecas
from git import Repo
import os, time
from datetime import date, datetime
import pandas as pd

In [98]:
# Funções
def set_path_atual_como_repositorio() -> Repo:
    repo_path = os.path.realpath('.')
    repo = Repo(repo_path)
    if not repo.bare: print(f"Repositório encontrado: {repo_path}")
    else: print("Erro: Diretório não contém um repositório Git.")
    return repo


def add_all_files_to_commit(repo:Repo) -> Repo:
    # Adicionar arquivos ao índice (staging area)
    repo.git.add(all=True)
    return repo


def commit_with_message(commit_message:str, repo:Repo) -> Repo:
    # Fazer commit
    repo.index.commit(commit_message)
    return repo


def git_push_all(repo:Repo) -> Repo:
    origin = repo.remote(name='origin')
    origin.push()
    return repo


def print_last_commits(repo:Repo, max:int=10) -> pd.DataFrame:
    prev_commits = list(repo.iter_commits(all=True, max_count=max))
    last_commits = []
    for x in range(len(prev_commits)):
        commit_item = {
            'data_commit': str(prev_commits[x].committed_datetime)[:10],
            'msg_commit': str(prev_commits[x].message),
            'autor_commit': str(prev_commits[x].author)
        }
        last_commits.append(commit_item)
    df = pd.DataFrame(last_commits)
    print(df)
    return df


def print_files_from_git(repo:Repo, level=0):
    root = repo.head.commit.tree
    for entry in root:
        print(f'{"-" * 4 * level}| {entry.path}')
        if entry.type == "tree":
            print_files_from_git(entry, level + 1)


def consultar_tags(repo:Repo):
    todas_tags = list(repo.tags)
    if len(todas_tags) > 0:
        ultima_tag = todas_tags[-1]
        print(ultima_tag)
    else:
        print('nenhuma tag criada')


def criar_tag_e_atualizar_online(repo:Repo, tag_version:str, tag_message:str) -> Repo:
    repo.git.execute(f'git tag -a {tag_version} -m "{tag_message}"')
    repo.git.execute(f'git push origin {tag_version}')
    return repo





In [ ]:
# Preparar ambiente
usuario_logado = 'edenilsonsantos' # os.getenv('username')
email = 'santoeen@gmail.com' # f'{usuario_logado}@gmail.com'
repo = set_path_atual_como_repositorio()
repo.git.execute(f'git config --global user.name "{usuario_logado}"')
repo.git.execute(f'git config --global user.email "{email}"')

try: repo.git.execute('git checkout -b main')
except: repo.git.checkout("main")
time.sleep(5)
try: repo.git.execute('git checkout -b homolog')
except: repo.git.checkout("homolog")

repo.git.execute(f'git config init.defaultBranch main')
repo.git.execute(f'git branch --set-upstream-to=origin/main main')
repo.git.execute(f'git branch --set-upstream-to=origin/homolog homolog')



In [87]:
# Add commit e Push All,  branch atual
repo = add_all_files_to_commit(repo)
repo = commit_with_message("Atualizando os arquivos com GitPython", repo)
repo.git.checkout("homolog")
time.sleep(3)
repo = git_push_all(repo)

In [99]:

consultar_tags(repo)


v1.0.0
